In [58]:
import os
import urllib
import rasterio as rio
import rasterio.plot

In [59]:
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
import planetary_computer as pc

In [60]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.011416, 48.616138],
            [-121.593718, 48.616138],
            [-121.593718, 48.912002],
            [-122.011416, 48.912002],
            [-122.011416, 48.616138],
        ]
    ],
}

In [115]:
time_of_interest = "2020-06-01/2020-08-01"

In [116]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["landsat-8-c2-l2"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)

# Check how many items were returned
items = list(search.get_items())
print(f"Returned {len(items)} Items")

Returned 1 Items


In [117]:
selected_item = sorted(items, key=lambda item: eo.ext(item).cloud_cover)[0]

print(
    f"Choosing {selected_item.id} from {selected_item.datetime.date()}"
    + f" with {selected_item.properties['eo:cloud_cover']}% cloud cover"
)

Choosing LC08_L2SP_047026_20200729_02_T1 from 2020-07-29 with 6.49% cloud cover


In [118]:
selected_item.id

'LC08_L2SP_047026_20200729_02_T1'

In [119]:
asset_id_list = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'reduced_resolution_browse']

In [120]:
imgdir = 'LS8_2020'
if not os.path.exists(imgdir):
    os.makedirs(imgdir)

In [121]:
import pystac
import planetary_computer
import rioxarray

item_url = f"https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-8-c2-l2/items/{selected_item.id}"

# Load the individual item metadata and sign the assets
item = pystac.Item.from_file(item_url)
signed_item = planetary_computer.sign(item)


for asset_id in asset_id_list:
    asset = item.assets[asset_id]
    print(asset.href)
    # NOTE: must 'sign' asset URLs before you can access them
    signed_asset = pc.sign_asset(asset)
    #print(signed_asset.href)
    #Local filename
    out_fn = os.path.join(imgdir, os.path.split(asset.href)[-1])
    #Check to see if file already exists
    if not os.path.exists(out_fn):
        print("Saving:", out_fn)
        #Download the file
        urllib.request.urlretrieve(signed_asset.href, out_fn)


https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2020/047/026/LC08_L2SP_047026_20200729_20200908_02_T1/LC08_L2SP_047026_20200729_20200908_02_T1_SR_B1.TIF
Saving: LS8_2020/LC08_L2SP_047026_20200729_20200908_02_T1_SR_B1.TIF
https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2020/047/026/LC08_L2SP_047026_20200729_20200908_02_T1/LC08_L2SP_047026_20200729_20200908_02_T1_SR_B2.TIF
Saving: LS8_2020/LC08_L2SP_047026_20200729_20200908_02_T1_SR_B2.TIF
https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2020/047/026/LC08_L2SP_047026_20200729_20200908_02_T1/LC08_L2SP_047026_20200729_20200908_02_T1_SR_B3.TIF
Saving: LS8_2020/LC08_L2SP_047026_20200729_20200908_02_T1_SR_B3.TIF
https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2020/047/026/LC08_L2SP_047026_20200729_20200908_02_T1/LC08_L2SP_047026_20200729_20200908_02_T1_SR_B4.TIF
Saving: LS8_2020/LC08_L2SP_047026_20200729_20200908_